# Onchain Attest Labels Using Python

In [ ]:
from web3 import Web3
from eth_abi.abi import encode
import json

# Connect to an Ethereum node
rpc = 'https://sepolia.base.org' # for Base use: https://mainnet.base.org
web3 = Web3(Web3.HTTPProvider(rpc))

# Check if connected to the node
if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum node")

# Load the EAS contract
from eas_contracts import eas_address, eas_abi
eas = web3.eth.contract(address=eas_address, abi=eas_abi)

# Import your wallet
address = '...'
private_key = '...'

# Label Pool Schema v1.0.0
schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'

# Input the label in OLI format:
#   chain_id should follow the CAIP-2 standard
#   contract_address should be the address of the contract you want to label
#   tags_json should be a JSON string
chain_id = 'eip:8453'
contract_address = '0x498581ff718922c3f8e6a244956af099b2652b2b'
tags_json = json.dumps(
                {'contract_name': 'Pool Manager v4',
                'project': 'PoolTogether',
                'is_eoa': False, 
                'deployment_tx': '0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349',
                'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
                'owner_project': 'uniswap',
                'version': 4,
                'deployment_date': '2025-01-21 20:28:43',
                'source_code_verified': 'https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/',
                'is_proxy': False}
            )

# ABI encode
encoded_data = encode(['string', 'string'], [chain_id, tags_json])
data = f"0x{encoded_data.hex()}"

# Create the transaction to call the attest function
transaction = eas.functions.attest({
    'schema': web3.to_bytes(hexstr=schema),
    'data': {
        'recipient': web3.to_checksum_address(contract_address),
        'expirationTime': 0,
        'revocable': True,
        'refUID': web3.to_bytes(hexstr='0x0000000000000000000000000000000000000000000000000000000000000000'),
        'data': web3.to_bytes(hexstr=data),
        'value': 0
    }
}).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1000000,
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

print(f'Transaction successful with hash: 0x{txn_hash.hex()}')

Transaction successful with hash: 0x3d5c0f1baec2d4b7aca7f694246def3c3572b0dda1cbd846a5a0181c7a907a59
